## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Databricks Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. This data will be imported in Databricks to create the following tables under the __`country_club`__ database.

<br>
1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into the Databricks platform to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the databricks platform, but for the final deliverable,
please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

### Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [1]:
%sql 
drop database if exists country_club cascade;
create database country_club;
show databases;

SyntaxError: expected 'else' after 'if' expression (4202119700.py, line 2)

### Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you. 
To get started, first upload the three CSV files to the DBFS as depicted in the following figure

![](https://i.imgur.com/QcCruBr.png)


Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [ ]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [ ]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
 |-- bookid: integer (nullable = true)
 |-- facid: integer (nullable = true)
 |-- memid: integer (nullable = true)
 |-- starttime: timestamp (nullable = true)
 |-- slots: integer (nullable = true)

Facilities Schema
root
 |-- facid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- membercost: double (nullable = true)
 |-- guestcost: double (nullable = true)
 |-- initialoutlay: integer (nullable = true)
 |-- monthlymaintenance: integer (nullable = true)

Members Schema
root
 |-- memid: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- address: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- telephone: string (nullable = true)
 |-- recommendedby: integer (nullable = true)
 |-- joindate: timestamp (nullable = true)



### Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [ ]:
permanent_table_name_bookings = "country_club.Bookings"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [ ]:
%sql
use country_club;
REFRESH table bookings;
REFRESH table facilities;
REFRESH table members;
show tables;

database,tableName,isTemporary
country_club,bookings,false
country_club,facilities,false
country_club,members,false


### Test a sample SQL query

__Note:__ You can use __`%sql`__ at the beginning of a cell and write SQL queries directly as seen in the following cell. Neat isn't it!

In [ ]:
%sql
select * from bookings limit 3

bookid,facid,memid,starttime,slots
0,3,1,2012-07-03T11:00:00.000+0000,2
1,4,1,2012-07-03T08:00:00.000+0000,2
2,6,0,2012-07-03T18:00:00.000+0000,2


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [ ]:
%sql
select facid, name from facilities where (membercost != 0) and (membercost IS NOT NULL);

facid,name
0,Tennis Court 1
1,Tennis Court 2
4,Massage Room 1
5,Massage Room 2
6,Squash Court


####  Q2: How many facilities do not charge a fee to members?

In [ ]:
%sql
select count(*) from facilities where (membercost = 0) or (membercost is NULL);

count(1)
4


out of curiosity: showing id and names of these facilities

In [ ]:

%sql
select facid, name from facilities where (membercost = 0) or (membercost IS NULL);

facid,name
2,Badminton Court
3,Table Tennis
7,Snooker Table
8,Pool Table


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [ ]:
%sql
select facid, name, membercost, monthlymaintenance, monthlymaintenance*0.2 as twenty_percent_mm from facilities where membercost < monthlymaintenance*0.2;

facid,name,membercost,monthlymaintenance,twenty_percent_mm
0,Tennis Court 1,5.0,200,40.0
1,Tennis Court 2,5.0,200,40.0
2,Badminton Court,0.0,50,10.0
3,Table Tennis,0.0,10,2.0
4,Massage Room 1,9.9,3000,600.0
5,Massage Room 2,9.9,3000,600.0
6,Squash Court,3.5,80,16.0
7,Snooker Table,0.0,15,3.0
8,Pool Table,0.0,15,3.0


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [ ]:
%sql
select * from facilities where facid in (1, 5);

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [ ]:
%sql
select name, 
monthlymaintenance, 
case 
when monthlymaintenance > 100 then "expensive"
else "cheap"
end
as labelled
from facilities;

name,monthlymaintenance,labelled
Tennis Court 1,200,expensive
Tennis Court 2,200,expensive
Badminton Court,50,cheap
Table Tennis,10,cheap
Massage Room 1,3000,expensive
Massage Room 2,3000,expensive
Squash Court,80,cheap
Snooker Table,15,cheap
Pool Table,15,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

the request for Q6 is a bit vague: I will be solving for 2 possible interpretations

1. interpretation 1: show members who have signed up to be members of country_club (assuming joindate is NULL when the member has not signed up yet)
2. interpretation 2: show members who have signed up to utilize facilites (if memid in bookings.memid)

In [ ]:
%sql
select memid, firstname, surname from members where joindate is not NULL;

memid,firstname,surname
0,GUEST,GUEST
1,Darren,Smith
2,Tracy,Smith
3,Tim,Rownam
4,Janice,Joplette
5,Gerald,Butters
6,Burton,Tracy
7,Nancy,Dare
8,Tim,Boothe
9,Ponder,Stibbons


In [ ]:
%sql
select m.memid, m.firstname, m.surname 
from members m 
where exists
(select b.memid 
from bookings b 
where m.memid = b.memid);

memid,firstname,surname
0,GUEST,GUEST
1,Darren,Smith
2,Tracy,Smith
3,Tim,Rownam
4,Janice,Joplette
5,Gerald,Butters
6,Burton,Tracy
7,Nancy,Dare
8,Tim,Boothe
9,Ponder,Stibbons


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

I have questions about Q7:
- Why is the 1st query missing 3 of the guests? the difference b/w 1st & 2nd queries is that "bookings.facid in (0, 1)" in the first's subquery is instead filtered by "facilities.facid in (0, 1)" of the second's main query
- How could I show non-duplicate members names and also the court name when there are multiple courts? Could I pull a view grouped by distinct member names? Not sure how I would do this. Did not discover.

In [ ]:
%sql
select CONCAT(members.firstname, ' ', members.surname) as member_name
from members 
where exists 
(select bookings.memid 
from bookings 
where members.memid = bookings.memid
and bookings.facid in (0, 1))
order by member_name;

member_name
Anne Baker
Burton Tracy
Charles Owen
Darren Smith
David Farrell
David Jones
David Pinker
Douglas Jones
Erica Crumpet
Florence Bader


In [ ]:
%sql
select distinct(CONCAT(members.firstname, ' ', members.surname)) as member_name,
facilities.name as name_of_court
from members 
join facilities 
where facilities.facid in (0, 1)
and exists
(select bookings.memid 
from bookings 
where members.memid = bookings.memid)
order by member_name;

member_name,name_of_court
Anna Mackenzie,Tennis Court 1
Anna Mackenzie,Tennis Court 2
Anne Baker,Tennis Court 1
Anne Baker,Tennis Court 2
Burton Tracy,Tennis Court 2
Burton Tracy,Tennis Court 1
Charles Owen,Tennis Court 2
Charles Owen,Tennis Court 1
Darren Smith,Tennis Court 1
Darren Smith,Tennis Court 2


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [ ]:
%sql
select b.bookid, 
f.name, 
CONCAT(m.firstname, ' ', m.surname) as member_name,
(case when (m.memid = 0) then f.guestcost else f.membercost end)*b.slots as total_cost
from bookings b 
join facilities f on b.facid = f.facid
join members m on b.memid = m.memid
where (cast(b.starttime as date) = date("2012-09-14"))
and ((case when (m.memid = 0) then f.guestcost else f.membercost end)*b.slots > 30)
order by total_cost desc;

bookid,name,member_name,total_cost
2946,Massage Room 2,GUEST GUEST,320.0
2937,Massage Room 1,GUEST GUEST,160.0
2940,Massage Room 1,GUEST GUEST,160.0
2942,Massage Room 1,GUEST GUEST,160.0
2926,Tennis Court 2,GUEST GUEST,150.0
2920,Tennis Court 1,GUEST GUEST,75.0
2922,Tennis Court 1,GUEST GUEST,75.0
2925,Tennis Court 2,GUEST GUEST,75.0
2948,Squash Court,GUEST GUEST,70.0
2941,Massage Room 1,Jemima Farrell,39.6


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [ ]:
%sql
select b.bookid, 
f.name, 
CONCAT(m.firstname, ' ', m.surname) as member_name,
(case when (m.memid = 0) then f.guestcost else f.membercost end)*b.slots as total_cost
from bookings b 
join facilities f on b.facid = f.facid
join members m on b.memid = m.memid
where ((case when (m.memid = 0) then f.guestcost else f.membercost end)*b.slots > 30)
and b.bookid in (select b.bookid from bookings b where (cast(b.starttime as date) = date("2012-09-14")))
order by total_cost desc;

bookid,name,member_name,total_cost
2946,Massage Room 2,GUEST GUEST,320.0
2937,Massage Room 1,GUEST GUEST,160.0
2940,Massage Room 1,GUEST GUEST,160.0
2942,Massage Room 1,GUEST GUEST,160.0
2926,Tennis Court 2,GUEST GUEST,150.0
2920,Tennis Court 1,GUEST GUEST,75.0
2922,Tennis Court 1,GUEST GUEST,75.0
2925,Tennis Court 2,GUEST GUEST,75.0
2948,Squash Court,GUEST GUEST,70.0
2941,Massage Room 1,Jemima Farrell,39.6


#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [ ]:
%sql
select sum((case when (b.memid = 0) then f.guestcost else f.membercost end)*b.slots) as revenue,
f.name
from bookings b, facilities f
where b.facid = f.facid
group by f.name
having (sum((case when (b.memid = 0) then f.guestcost else f.membercost end)*b.slots) < 1000)
order by revenue;

revenue,name
180.0,Table Tennis
240.0,Snooker Table
270.0,Pool Table


In [ ]:
# the following cell shows total revenues per each facilities

In [ ]:
%sql
select sum((case when (b.memid = 0) then f.guestcost else f.membercost end)*b.slots) as revenue,
f.name
from bookings b, facilities f
where b.facid = f.facid
group by f.name
order by revenue;

revenue,name
180.0,Table Tennis
240.0,Snooker Table
270.0,Pool Table
1906.5,Badminton Court
13468.0,Squash Court
13860.0,Tennis Court 1
14310.0,Tennis Court 2
14454.599999999995,Massage Room 2
50351.600000000275,Massage Room 1
